**Initial Model Test with pretrained Resnet models**

Change log level

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

Imports

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
import keras_tuner as kt

Load data

In [3]:
test_data_directory = 'data/test'
train_data_directory = 'data/train'
valid_data_directory = 'data/valid'
batch_size = 32

def data_loader(directory, batch_size):
    ds = image_dataset_from_directory(
        directory = directory,
        seed = 1324,
        label_mode = 'categorical',
        image_size = (350, 350),
        batch_size = batch_size
    )  
    return ds

train_ds = data_loader(train_data_directory, batch_size)

test_ds = data_loader(test_data_directory, batch_size)

valid_ds = data_loader(valid_data_directory, batch_size)

Found 30247 files belonging to 2 classes.
Found 6298 files belonging to 2 classes.
Found 6300 files belonging to 2 classes.


Build model funtion for keras_tuner

In [4]:
def build_model(hp):
    # initialize model
    model = Sequential()

    # import pretrained model
    resnet_model = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=(350,350,3),
        pooling='avg',
        weights='imagenet'
    )

    # exclude pretrained model weights from being recalculated
    for layer in resnet_model.layers:
        layer.trainable = False

    # add pretrained ResNet50 model to sequential model
    model.add(resnet_model)

    # add additional layers to model
    model.add(Flatten())

    model.add(Dense(
       # Tune number of units.
        units=hp.Choice("units", values=[64,128,256,512]),
        # Tune the activation function to use.
        activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )


    model.add(Dense(2, tf.nn.softmax))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

Define Tuner

In [5]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    seed=1234,
    overwrite=True,
    max_trials=20,
    directory='models/hp_tuning')

Run tuner HP search

In [8]:
tuner.search(train_ds, epochs=5, validation_data=(valid_ds))

Trial 8 Complete [00h 04m 25s]
val_accuracy: 0.9868254065513611

Best val_accuracy So Far: 0.986984133720398
Total elapsed time: 00h 47m 32s
INFO:tensorflow:Oracle triggered exit


In [9]:
best_model = tuner.get_best_models()[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


In [10]:
best_hps.get('activation')

'relu'

In [11]:
best_hps.get('units')

265

In [12]:
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 265)               542985    
                                                                 
 dense_1 (Dense)             (None, 2)                 532       
                                                                 
Total params: 24,131,229
Trainable params: 543,517
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
score = best_model.evaluate(test_ds)

197/197 [==============================] - 10s 48ms/step - loss: 0.0321 - accuracy: 0.9879
